# Binary LSTM classification model

In this notebook, we've replicated the binary LSTM model for the DGA/non-DGA classification of a domain name; from the Endgame paper:

"Predicting Domain Generation Algorithms with Long Short-Term Memory Networks"
http://arxiv.org/abs/1611.00791v1


In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer

from tensorflow.python.client import device_lib

In [52]:
# Confirm gpu is being picked up for acceleration
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8874625246908832808
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3183759360
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14828915549369756570
physical_device_desc: "device: 0, name: GeForce GTX 780M, pci bus id: 0000:01:00.0, compute capability: 3.0"
]


In [53]:
# Read DGA and Cisco high confidence data
dga_df = pd.read_csv('..\\data\\2018_0923\\dga-feed-high.csv', header=None, skiprows=15)
cisco_df = pd.read_csv('..\\data\\2018_0923\\top-1m.csv', header=None)

In [54]:
# display head
def display_df(dga_df_, cisco_df_):
    display("DGA feed sample: {}".format( dga_df_.shape) )
    display(dga_df_.head())
    display("Cisco feed sample: {}".format( cisco_df_.shape) )
    display(cisco_df_.head())

In [55]:
# Remove unused columns, add output label 'dga'

dga_df_slim =   dga_df.drop(columns=range(1,dga_df.shape[1]), inplace=False)
dga_df_slim.columns = ['domain']
cisco_df_slim = cisco_df.drop(columns=[0], inplace=False)
cisco_df_slim.columns = ['domain']
dga_df_slim['dga'] = 1
cisco_df_slim['dga'] = 0

display_df(dga_df_slim, cisco_df_slim)
unified_df = pd.concat([cisco_df_slim, dga_df_slim], ignore_index=True)

'DGA feed sample: (381953, 2)'

,domain,dga
0,plvklpgwivery.com,1
1,dnuxdhcgblsgy.net,1
2,qjlullhfkiowp.biz,1
3,elkidddodxdly.ru,1
4,rnbfwuprlwfor.org,1


'Cisco feed sample: (1000000, 2)'

,domain,dga
0,netflix.com,0
1,api-global.netflix.com,0
2,prod.netflix.com,0
3,push.prod.netflix.com,0
4,google.com,0


In [56]:
# Separate input sequences (domains) and output labels (DGA 0/1), and do train/test split

X = unified_df['domain']
Y = unified_df['dga']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=23)

In [60]:
# Binary classification LSTM model

max_features = 1000                                              # length of vocabulary
batch_size = 128                                                 # input batch size
num_epochs = 5                                                   # epochs to train

# encode string characters to integers
encoder = Tokenizer(num_words=500, char_level=True)
encoder.fit_on_texts(X_train)                                    # build character indices
X_train_tz = encoder.texts_to_sequences(X_train)

# Model definition - this is the core model from Endgame
model=Sequential()
model.add(Embedding(max_features, 128, input_length=75))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# Pad sequence where sequences are case insensitive characters encoded to
# integers from 0 to number of valid characters
X_train_pad=sequence.pad_sequences(X_train_tz, maxlen=75)

# Train where Y_train is 0-1
model.fit(X_train_pad, Y_train, batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
1105562/1105562 [==============================] - 1339s 1ms/step - loss: 0.0647
Epoch 2/5
1105562/1105562 [==============================] - 1351s 1ms/step - loss: 0.0401
Epoch 3/5
1105562/1105562 [==============================] - 1406s 1ms/step - loss: 0.0353
Epoch 4/5
1105562/1105562 [==============================] - 1510s 1ms/step - loss: 0.0328
Epoch 5/5
1105562/1105562 [==============================] - 1537s 1ms/step - loss: 0.0313


In [61]:
# Validation on test dataset

X_test_pad = sequence.pad_sequences(encoder.texts_to_sequences(X_test), maxlen=75)
Y_pred = model.predict_classes(X_test_pad)
acc = accuracy_score(Y_test, Y_pred)
print("Model accuracy = {:8.3f} %".format(acc*100))

Model accuracy =   99.031 %


## Look ahead and next steps:
__1__ Look closer at the misclassified domains. Any particular DGA category stands out? What do we need to improve? 

__2__ Improving classification accuracy - more balanced dataset especially for the multiclass classification.

__3__ Learning from scratch takes significant time. Need to implement model update in batches of new domain dataset.

__4__ Modify the model to do multiclass classification across the various DGA categories. Do we need to trim down the categories - dataset shows 60+ categories and new ones may be added any time.